In [1]:
import yfinance as yf
import pandas as pd
import statsmodels.api as sm
from datetime import timedelta
from datetime import datetime
import requests
import zipfile 
import os

Extrair DFP (Demosntrações Financeira Padronizadas) da CVM - Resultados Anuais

In [2]:
#pega o diretorio atual
#diretorio_atual = os.getcwd()

#definir caminho para salvar os arquivos -- editar aqui caso queira mudar
os.chdir(f"C:\workspace_mestrado\dados_cvm_anuais")

#pega dados desde 2010 --- editar aqui para pegar mais anos
anos = range(2010, 2025)
url_base = "http://dados.cvm.gov.br//dados/CIA_ABERTA/DOC/DFP/DADOS/"

for ano in anos:
    download = requests.get(url_base + f"dfp_cia_aberta_{ano}.zip")
    #salvar o arquivo zip no caminho C:\workspace_mestrado\dados_cvm_anuais
    open(f"dfp_cia_aberta_{ano}.zip", "wb").write(download.content)
    print(f"Arquivo {ano} baixado e salvo")
    

Arquivo 2010 baixado e salvo
Arquivo 2011 baixado e salvo
Arquivo 2012 baixado e salvo
Arquivo 2013 baixado e salvo
Arquivo 2014 baixado e salvo
Arquivo 2015 baixado e salvo
Arquivo 2016 baixado e salvo
Arquivo 2017 baixado e salvo
Arquivo 2018 baixado e salvo
Arquivo 2019 baixado e salvo
Arquivo 2020 baixado e salvo
Arquivo 2021 baixado e salvo
Arquivo 2022 baixado e salvo
Arquivo 2023 baixado e salvo
Arquivo 2024 baixado e salvo


Juntar arquivos separados em um único

In [3]:
lista_demonstracoes_2010_2022 = []
#definir caminho para salvar os arquivos -- editar aqui caso queira mudar
path = "C:\\workspace_mestrado\\dados_cvm_anuais\\"

for arquivo in os.listdir(f"{path}"):
    if arquivo.endswith(".zip"):
        with zipfile.ZipFile(arquivo, "r") as zipped:
            zipped.extractall()
            print(f"Arquivo {arquivo} extraído")
            for arquivo_csv in zipped.namelist():
                if arquivo_csv.endswith(".csv") and "DRE" in arquivo_csv and "con" in arquivo_csv:
                    #pegar apenas os arquivos que tiverem DRE no nome
                    demonstracao = pd.read_csv(arquivo_csv, sep=";", decimal=",", encoding="ISO-8859-1", dtype= {"ORDEM_EXERC": "category"})
                    lista_demonstracoes_2010_2022.append(demonstracao)
                    
                    print(f"*Arquivo {arquivo_csv} adicionado à lista de demonstrações")
                os.remove(arquivo_csv)        


Arquivo dfp_cia_aberta_2010.zip extraído
*Arquivo dfp_cia_aberta_DRE_con_2010.csv adicionado à lista de demonstrações
Arquivo dfp_cia_aberta_2011.zip extraído
*Arquivo dfp_cia_aberta_DRE_con_2011.csv adicionado à lista de demonstrações
Arquivo dfp_cia_aberta_2012.zip extraído
*Arquivo dfp_cia_aberta_DRE_con_2012.csv adicionado à lista de demonstrações
Arquivo dfp_cia_aberta_2013.zip extraído
*Arquivo dfp_cia_aberta_DRE_con_2013.csv adicionado à lista de demonstrações
Arquivo dfp_cia_aberta_2014.zip extraído
*Arquivo dfp_cia_aberta_DRE_con_2014.csv adicionado à lista de demonstrações
Arquivo dfp_cia_aberta_2015.zip extraído
*Arquivo dfp_cia_aberta_DRE_con_2015.csv adicionado à lista de demonstrações
Arquivo dfp_cia_aberta_2016.zip extraído
*Arquivo dfp_cia_aberta_DRE_con_2016.csv adicionado à lista de demonstrações
Arquivo dfp_cia_aberta_2017.zip extraído
*Arquivo dfp_cia_aberta_DRE_con_2017.csv adicionado à lista de demonstrações
Arquivo dfp_cia_aberta_2018.zip extraído
*Arquivo dfp_ci

Ajustar base de dados

In [4]:
#agrupa os itens da lista em um dataframe
base_dados = pd.concat(lista_demonstracoes_2010_2022)

base_dados[['con_ind', 'tipo_dem']] = base_dados['GRUPO_DFP'].str.split(' - ', expand=True)
base_dados['tipo_dem'] = base_dados['tipo_dem'].str.strip()
base_dados = base_dados[base_dados['ORDEM_EXERC'] != 'PENÚLTIMO']

#remoção de colunas que não serão utilizadas
base_dados = base_dados.drop(columns=['CNPJ_CIA', 'MOEDA', 'ORDEM_EXERC', 'con_ind', 'GRUPO_DFP', 'tipo_dem'])


base_dados['DT_REFER'] = pd.to_datetime(base_dados['DT_REFER'], format='ISO8601')
base_dados['DT_INI_EXERC'] = pd.to_datetime(base_dados['DT_INI_EXERC'], format='ISO8601')
base_dados['DT_FIM_EXERC'] = pd.to_datetime(base_dados['DT_FIM_EXERC'], format='ISO8601')

#transformar em csv
base_dados.to_csv('C:\\workspace_mestrado\\dados_cvm_anuais\\base_dados_2024.csv', sep=";")
print("Arquivo 'base_dados_2024.csv' salvo com sucesso!")

Arquivo salvo com sucesso!


In [5]:
#listar empresas no arquivo
lista_cias = base_dados[['DENOM_CIA', 'CD_CVM']]
lista_cias = lista_cias.drop_duplicates()

#transformar salvar arquivo em csv
lista_cias = pd.DataFrame(lista_cias)
lista_cias.to_csv('C:\\workspace_mestrado\\lista_cias.csv', sep=";")
print("Arquivo 'lista_cias.csv' salvo com sucesso!")

Arquivo salvo com sucesso!
